# Level 1: Loading and Exploring Data

It's time to dive into the world of data science! With the growing collection of data in many different fields, data science has become an important tool in unlocking business value, developing a better understanding of situations, and solving problems in a principled fashion. We hope you come to enjoy and continue on to explore data science through this track. 

This track will gently acquaint you with how people get started with a data problem; in particular, we will cover obtaining data, how data is usually stored, getting data into usable form in Python, doing exploratory work, and correcting for missing or unrecorded values.

## Obtaining Data
Before starting to work on a data problem, we need to actually get some data to analyze! Sometimes, this will be given to you directly, but often times, you'll have to go look for data that matches the problem you want to analyze. The good news is that there are an ever growing number of datasets you can find on the Internet! One of our favorite type of sources is government open data, which has information on all kinds of things that you both can and can't imagine. For the purpose of this level, we'll be using information on NYC Green Taxi trips during January 2016. You can find more information on this dataset [here](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml), and the actual data comes from [this link](https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2016-01.csv). For your convenience, it's already in the `datasets` folder as `green_tripdata_2016-01.csv`. You might find it helpful to go through [this data dictionary](http://www.nyc.gov/html/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf) later if there are columns that don't make sense to you.

## CSV Format
Let's now understand how this data is stored; while there are other formats such as JSON, CSV, or comma-separated values, files are one of the most common file formats for data scientists. We'll look at a toy example about fake Columbia library traffic to familiarize ourselves:
`date,library,number_students
1-16-2017,butler,0
1-16-2017,noco,0
1-16-2017,watson,0
1-17-2017,butler,100
1-17-2017,noco,55
1-17-2017,watson,67
1-18-2017,butler,110
1-18-2017,noco,48
1-18-2017,watson,78
1-19-2017,butler,35
1-19-2017,noco,24
1-19-2017,watson,18`
The first thing to take note of is that all the different values are separated by commas: hence the name CSV. Another thing to notice is that the column names are given in the first row. The last important thing to note about CSV files is that each row is on its own line. There's not much more to CSV files than these few points, and their convenience makes them a file format of choice for data scientists.

## Importing Data into Python
Now that we understand some of the basics of the data, let's try playing around with it in Python. One typical way to work with datasets is using `pandas`, which is a package which makes it easy to manipulate data stored in tabular form or as `pandas` more succintly says: dataframes. Let's load up `pandas`.

In [4]:
import pandas as pd

It's also usually useful to have `numpy` around, and we'll use it later on.

In [5]:
import numpy as np

Let's now load our dataset using the `read_csv` function; it does pretty much exactly what the name implies: read data from a CSV and return the resulting dataframe.

In [6]:
taxi_data = pd.read_csv("datasets/green_tripdata_2016-01.csv")

Great, we now have our data in Python. Let's start to explore it; the first thing we should check out is how much data we have in terms of the number of rows and columns.

In [7]:
number_rows = len(taxi_data.index)
number_columns = len(taxi_data.columns)
print("Number of rows: %d" % number_rows)
print("Number of columns: %d" % number_columns)

Number of rows: 1445285
Number of columns: 21


Wowza! 1.4 million rows or so, guess we're dealing with big data. :) Another interesting thing to check out is what kinds of columns are included in the dataset: we can do this by checking out the `columns` attribute.

In [8]:
print(taxi_data.columns)

Index(['VendorID', 'lpep_pickup_datetime', 'Lpep_dropoff_datetime',
       'Store_and_fwd_flag', 'RateCodeID', 'Pickup_longitude',
       'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude',
       'Passenger_count', 'Trip_distance', 'Fare_amount', 'Extra', 'MTA_tax',
       'Tip_amount', 'Tolls_amount', 'Ehail_fee', 'improvement_surcharge',
       'Total_amount', 'Payment_type', 'Trip_type '],
      dtype='object')


There's a lot we can learn from just the columns that are stored: for example, we can get information like when the trip started and ended, where the trip started and ended, whether the people tipped, and how much the fare was.

We can also investigate what types of data the columns have.

In [9]:
taxi_data.dtypes

VendorID                   int64
lpep_pickup_datetime      object
Lpep_dropoff_datetime     object
Store_and_fwd_flag        object
RateCodeID                 int64
Pickup_longitude         float64
Pickup_latitude          float64
Dropoff_longitude        float64
Dropoff_latitude         float64
Passenger_count            int64
Trip_distance            float64
Fare_amount              float64
Extra                    float64
MTA_tax                  float64
Tip_amount               float64
Tolls_amount             float64
Ehail_fee                float64
improvement_surcharge    float64
Total_amount             float64
Payment_type               int64
Trip_type                float64
dtype: object

Oops, looks like `pandas` isn't recognizing the times as such because it just labels it as `object`, which usually means string. Let's do that conversion.

In [10]:
taxi_data.lpep_pickup_datetime = pd.to_datetime(taxi_data.lpep_pickup_datetime)
taxi_data.Lpep_dropoff_datetime = pd.to_datetime(taxi_data.Lpep_dropoff_datetime)
taxi_data.dtypes

VendorID                          int64
lpep_pickup_datetime     datetime64[ns]
Lpep_dropoff_datetime    datetime64[ns]
Store_and_fwd_flag               object
RateCodeID                        int64
Pickup_longitude                float64
Pickup_latitude                 float64
Dropoff_longitude               float64
Dropoff_latitude                float64
Passenger_count                   int64
Trip_distance                   float64
Fare_amount                     float64
Extra                           float64
MTA_tax                         float64
Tip_amount                      float64
Tolls_amount                    float64
Ehail_fee                       float64
improvement_surcharge           float64
Total_amount                    float64
Payment_type                      int64
Trip_type                       float64
dtype: object

Ah, much better. Now that we have some idea of what we'll be looking at, let's look at a few examples of trips. We'll do this using the `head` function, which will show us the first 5 rows. 

(Note that you could do `print(taxi_data.head())`, but the HTML table formatting that Jupyter lets us do looks much cleaner. To use the automatic formatting, just have the call to `head` be the last thing in your cell.)

In [40]:
taxi_data.head()

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,improvement_surcharge,Total_amount,Payment_type,Trip_type
0,2,2016-01-01 00:29:24,2016-01-01 00:39:36,N,1,-73.928642,40.680611,-73.924278,40.698044,1,...,8.0,0.5,0.5,1.86,0.0,NaN,0.3,11.16,1,1.0
1,2,2016-01-01 00:19:39,2016-01-01 00:39:18,N,1,-73.952675,40.723175,-73.923920,40.761379,1,...,15.5,0.5,0.5,0.00,0.0,NaN,0.3,16.80,2,1.0
2,2,2016-01-01 00:19:33,2016-01-01 00:39:48,N,1,-73.971611,40.676105,-74.013161,40.646072,1,...,16.5,0.5,0.5,4.45,0.0,NaN,0.3,22.25,1,1.0
3,2,2016-01-01 00:22:12,2016-01-01 00:38:32,N,1,-73.989502,40.669579,-74.000648,40.689034,1,...,13.5,0.5,0.5,0.00,0.0,NaN,0.3,14.80,2,1.0
4,2,2016-01-01 00:24:01,2016-01-01 00:39:22,N,1,-73.964729,40.682854,-73.940720,40.663013,1,...,12.0,0.5,0.5,0.00,0.0,NaN,0.3,13.30,2,1.0


We could also just as easily look at the last 5 rows with the `tail` function.

In [12]:
taxi_data.tail()

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,improvement_surcharge,Total_amount,Payment_type,Trip_type
1445280,2,2016-01-31 23:59:36,2016-02-01 00:07:53,N,1,-73.807716,40.700451,-73.768478,40.698082,1,...,10.5,0.5,0.5,0.00,0.0,NaN,0.3,11.80,2,1.0
1445281,2,2016-01-31 23:26:06,2016-01-31 23:31:22,N,1,-73.951408,40.714001,-73.941483,40.724766,2,...,6.0,0.5,0.5,1.00,0.0,NaN,0.3,8.30,1,1.0
1445282,2,2016-01-31 23:50:36,2016-01-31 23:56:19,N,1,-73.954842,40.730213,-73.944626,40.726303,2,...,6.5,0.5,0.5,2.34,0.0,NaN,0.3,10.14,1,1.0
1445283,2,2016-01-31 23:57:47,2016-02-01 00:06:58,N,1,-73.953491,40.817211,-73.977127,40.779583,1,...,10.5,0.5,0.5,1.50,0.0,NaN,0.3,13.30,1,1.0
1445284,2,2016-01-31 23:59:54,2016-02-01 00:07:06,N,1,-73.892578,40.747250,-73.910049,40.775539,1,...,10.0,0.5,0.5,2.82,0.0,NaN,0.3,14.12,1,1.0


It seems like we have a lot of columns, so some of the columns are getting cut off; we can easily subset to have less columns by using indexing.

In [13]:
location_information = taxi_data[["Pickup_longitude", "Pickup_latitude", "Dropoff_longitude", "Dropoff_latitude", "Trip_distance"]]
location_information.head()

,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Trip_distance
0,-73.928642,40.680611,-73.924278,40.698044,1.46
1,-73.952675,40.723175,-73.923920,40.761379,3.56
2,-73.971611,40.676105,-74.013161,40.646072,3.79
3,-73.989502,40.669579,-74.000648,40.689034,3.01
4,-73.964729,40.682854,-73.940720,40.663013,2.55


We can also look at any arbitrary set of rows in a similar way.

In [14]:
taxi_data[2014:2018]

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,improvement_surcharge,Total_amount,Payment_type,Trip_type
2014,2,2016-01-01 00:49:38,2016-01-01 01:01:55,N,1,-73.911057,40.768673,-73.849457,40.724583,1,...,19.0,0.5,0.5,0.0,0.0,NaN,0.3,20.3,2,1.0
2015,2,2016-01-01 00:57:12,2016-01-01 01:02:13,N,1,-73.880302,40.828606,-73.895561,40.821033,1,...,6.0,0.5,0.5,0.0,0.0,NaN,0.3,7.3,2,1.0
2016,2,2016-01-01 00:57:17,2016-01-01 01:01:33,N,1,-73.869339,40.748859,-73.863525,40.737297,1,...,5.5,0.5,0.5,0.0,0.0,NaN,0.3,6.8,2,1.0
2017,2,2016-01-01 00:52:10,2016-01-01 01:02:17,N,1,-73.965530,40.683224,-73.943916,40.683205,1,...,8.0,0.5,0.5,0.0,0.0,NaN,0.3,9.3,1,1.0


Besides extracting sets of columns, we can also get a single column by just using its name (assuming it has no spaces or special characters); here's an example:

In [15]:
distances = taxi_data.Trip_distance
distances[:5]

0    1.46
1    3.56
2    3.79
3    3.01
4    2.55
Name: Trip_distance, dtype: float64

We can also perform common summary operations very easily using `pandas`; for example, let's have it summarize the column representing the total amount.

In [16]:
taxi_data.Total_amount.describe()

count    1.445285e+06
mean     1.441838e+01
std      1.201146e+01
min     -4.975000e+02
25%      7.800000e+00
50%      1.116000e+01
75%      1.730000e+01
max      1.000800e+03
Name: Total_amount, dtype: float64

Something should stand out like a sore thumb: some trip cost negative ~500 dollars! To simplify our dataset, let's just get rid of all the trips that had nonpositive total amount using conditional indexing where each row is kept or not based on a boolean condition.

In [17]:
sensible_trips = taxi_data[taxi_data.Total_amount > 0]
print("New number of rows: %d" % len(sensible_trips.index))
print("Percentage of trips with negative total amount: %.2f%%" % \
      (100 * (1.0 - float(len(sensible_trips.index)) / len(taxi_data.index))))

New number of rows: 1438657
Percentage of trips with negative total amount: 0.46%


Thank goodness that most of our trips don't have this weird behavior.

Let's try to make a new column looking at what percentage of the total fare was from the trip; this is easy to do in `pandas`.

In [18]:
sensible_trips.tip_percentage = sensible_trips.Tip_amount / sensible_trips.Total_amount
sensible_trips.tip_percentage.describe()

count    1.438657e+06
mean     7.098524e-02
std      9.227385e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.666667e-01
max      1.000000e+00
dtype: float64

From this simple calculation, we can say that a tip isn't given in over half of rides since the median value is 0; that's pretty weird, but it's cool that we were able to discover that with just a few `pandas` operations!

We can also do pretty cool statistics using the `groupby` function; essentially, it lets you specify groups of rows based on a variable, and then we can do statistics on each group. This lets us do things like explore whether the tip amount is related to the number of people who were in the car.

In [19]:
means = sensible_trips.tip_percentage.groupby(sensible_trips.Passenger_count).mean()
stds = sensible_trips.tip_percentage.groupby(sensible_trips.Passenger_count).std()
pd.DataFrame({'mean' : means, 'std. dev.' : stds})

,mean,std. dev.
Passenger_count,,
0,0.053812,0.094425
1,0.070866,0.092231
2,0.071527,0.093398
3,0.074753,0.094416
4,0.065178,0.094548
5,0.070330,0.090389
6,0.073297,0.089202
7,0.045288,0.079779
8,0.029817,0.059235


That's awkward.. There were some rides where 0 people were in the taxi! Anyhow, there doesn't seem to be a whole lot of difference in the mean across different passenger counts given the variation in those values (as indicated by the standard deviation).

## Missing Values

Our taxi data was super nice in many ways: among them, it had didn't have any missing values. In this next part, we'll explore what to do if we do have missing values in our data. Missing values can come up for any number of reasons such as sensor failure or a value not being applicable for a particular row; traditionally, they are represented as a special character such as `?` or `NA` in a CSV file though sometimes the value could just be blank. It's often a good practice to look at a CSV if you suspect it has missing values to see what the convention is as there is no standard.

To get a CSV with missing values, we'll be using a different dataset the [Air Quality](http://archive.ics.uci.edu/ml/datasets/Air+Quality) dataset from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.html). These datasets come in handy when you need any suitable set of data to practice your machine learning rather than explore through data science. It is already located in your `datasets` folder as `AirQualityUCI.csv`, so let's load it up.

In [22]:
air_quality_data = pd.read_csv("datasets/AirQualityUCI.csv", sep = ";")
air_quality_data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN


The first thing that should jump out is those unnamed columns at the end, let's dig deeper. We'll use the `isnan` function from `numpy` to check if there are any real values in the column and the `all` function also from `numpy` to quickly tell us if all the values are junk.

In [28]:
all_nan_15 = np.all(np.isnan(air_quality_data["Unnamed: 15"].values))
all_nan_16 = np.all(np.isnan(air_quality_data["Unnamed: 16"].values))
print("All Garbage in 15? %s" % all_nan_15)
print("All Garbage in 16? %s" % all_nan_16)

All Garbage in 15? True
All Garbage in 16? True


Since all the values are not usable, let's go ahead and drop those columns using the `drop` function. Note that it's best practice to define a new dataframe without those columns rather than deleting them from the original dataframe because running your code twice would fail (you can't delete them again after having deleted them once). Also, the `axis = 1` just means drop a column rather than drop a row.

In [31]:
clean_air_quality_data = air_quality_data.drop(["Unnamed: 15", "Unnamed: 16"], axis = 1)
clean_air_quality_data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578"
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255"
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502"
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867"
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888"


Okay, now let's go looking for those missing values. We'll use the `isnull` function from `pandas`, which would produce a new data frame where every value has been replaced by whether it's junk or not. We can then summarize it by using the `sum` function, which by default does column sums.

In [32]:
clean_air_quality_data.isnull().sum()

Date             114
Time             114
CO(GT)           114
PT08.S1(CO)      114
NMHC(GT)         114
C6H6(GT)         114
PT08.S2(NMHC)    114
NOx(GT)          114
PT08.S3(NOx)     114
NO2(GT)          114
PT08.S4(NO2)     114
PT08.S5(O3)      114
T                114
RH               114
AH               114
dtype: int64

Interesting, it isn't the case that a certain variable is missing more than others, so a good guess would be that 114 rows have no values for each column. Let's check out a few.

In [33]:
clean_air_quality_data[clean_air_quality_data.Date.isnull()].head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
9357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Just as we had suspected! (Note that we had to check the assumption that the missing values were just located in the same 114 rows.) We'll try two different approaches to fixing this issue: getting rid of the problematic rows and imputing (read: replacing, imputing is just fancy talk in the business) the missing values with the previous non-missing value. The second is a good strategy since this is time-series data, but if the observations were unconnected, we might use the mean of the column instead.

First up is getting rid of the problematic rows.

In [38]:
strictly_clean_air_quality_data = clean_air_quality_data[np.logical_not(clean_air_quality_data.Date.isnull())]
print(strictly_clean_air_quality_data.isnull().sum())
strictly_clean_air_quality_data.head()

Date             0
Time             0
CO(GT)           0
PT08.S1(CO)      0
NMHC(GT)         0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH               0
dtype: int64


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578"
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255"
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502"
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867"
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888"


Awesome! Let's try the other way now. `pandas` makes it super easy for us.

In [43]:
imputed_air_quality_data = clean_air_quality_data.fillna(method = "ffill")
imputed_air_quality_data[9355:9360]

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
9355,04/04/2005,13.00.00,"2,1",1003.0,-200.0,"9,5",961.0,235.0,702.0,156.0,1041.0,770.0,"28,3","13,5","0,5139"
9356,04/04/2005,14.00.00,"2,2",1071.0,-200.0,"11,9",1047.0,265.0,654.0,168.0,1129.0,816.0,"28,5","13,1","0,5028"
9357,04/04/2005,14.00.00,"2,2",1071.0,-200.0,"11,9",1047.0,265.0,654.0,168.0,1129.0,816.0,"28,5","13,1","0,5028"
9358,04/04/2005,14.00.00,"2,2",1071.0,-200.0,"11,9",1047.0,265.0,654.0,168.0,1129.0,816.0,"28,5","13,1","0,5028"
9359,04/04/2005,14.00.00,"2,2",1071.0,-200.0,"11,9",1047.0,265.0,654.0,168.0,1129.0,816.0,"28,5","13,1","0,5028"


This seems to be doing okay since the previously missing rows now take the values from row with index number 9356, but of course, we could always do better. For example, the `Date` column probably should still increment! `pandas` provides us a few interpolation options that would account for things like this, but you should always make sure that it's doing something sensible by checking known missing value cases!